# Data Analysis

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from scipy.stats import norm, genextreme
import numpy as np
from datetime import datetime
from data import get_data, extract_date_data
from utils import get_path, load_model_data, month, day_of_week, precip_type
from sklearn.model_selection import ShuffleSplit
from tqdm.auto import tqdm
plt.style.use('ggplot')

## Feature selection

We start by loading in the weather features and choosing a (roughly) independent subset of these to use for training our model.

In [ ]:
weather_df = pd.read_csv(get_path('data') / 'weather_data.tsv', sep = '\t')
weather_df['precip_type'] = weather_df['precip_type'].map(precip_type)
weather_df.head()

In [ ]:
feat_df = weather_df.copy()[[col for col in list(weather_df.columns) if col not in ['date']]]
f = plt.figure(figsize = (13, 10))
plt.matshow(feat_df.corr(), fignum = f.number, cmap = 'coolwarm', vmin = -1, vmax = 1)
plt.xticks(range(feat_df.shape[1]), feat_df.columns, fontsize=14, rotation=90)
plt.yticks(range(feat_df.shape[1]), feat_df.columns, fontsize=14)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)

As we can see on the above heatmap, our weather features are highly dependent, so we *do* need to trim these down a bit.

We see that the majority of the dependencies cluster in three groups: precipitation, wind speed / gusts, and temperature. We therefore start by only including the average values of these three groups, as we hypothesise that these averages will be more reliable when we fetch forecasts for our predictions.

In [ ]:
features = ['precip_intensity_avg', 'precip_type', 'wind_speed_avg', 'gust_avg', 'temp_avg', 'humidity']
trimmed_feat_df = weather_df.copy()[features]
f = plt.figure(figsize = (9, 6))
plt.matshow(trimmed_feat_df.corr(), fignum = f.number, cmap = 'coolwarm', vmin = -1, vmax = 1)
plt.xticks(range(trimmed_feat_df.shape[1]), trimmed_feat_df.columns, fontsize=14, rotation=90)
plt.yticks(range(trimmed_feat_df.shape[1]), trimmed_feat_df.columns, fontsize=14)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)

The major dependency remaining is between wind speed average and gust average, so we remove the gust average.

This is again because we hypothesise that wind speeds are easier to forecast.

In [ ]:
features = ['precip_intensity_avg', 'precip_type', 'wind_speed_avg', 'temp_avg', 'humidity']
final_feat_df = weather_df.copy()[features]
f = plt.figure(figsize = (8, 5))
plt.matshow(final_feat_df.corr(), fignum = f.number, cmap = 'coolwarm', vmin = -1, vmax = 1)
plt.xticks(range(final_feat_df.shape[1]), final_feat_df.columns, fontsize=14, rotation=90)
plt.yticks(range(final_feat_df.shape[1]), final_feat_df.columns, fontsize=14)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)

This looks good! We'll also be using some date-related features: `month`, `day_of_month` and `day_of_week`.

Let's load in our training dataset with all these features.

In [ ]:
X, y = get_data(include_date = True)
df = X.copy()
X.drop(columns = ['date'], inplace = True)
df['total'] = y.copy()
df.head()

## Analysis of target labels

In [ ]:
def earlier_than(date1: str, date2: str):
    from datetime import datetime
    date1, date2 = datetime.strptime(date1, '%Y-%m-%d'), datetime.strptime(date2, '%Y-%m-%d')
    return date1 < date2

Now that we've chosen our features, let's have a look at what we're trying to predict.

Here's a plot of the reported total demand for a Bristol Soup Run Trust (BSRT) over time.

In [ ]:
xticks = list(range(1, len(df.index), 10))
ax = df.total.plot(figsize = (15, 7), title = 'Total BSRT demand over time', 
                   rot = 45, xticks = xticks, color = 'g')
ax.set_xticklabels(df.date[xticks])
plt.show()

We also see that these values approximately follow a normal distribution, as can be seen on the following histogram.

In [ ]:
train_idxs = [idx for idx, date in zip(df.index, df.date) if earlier_than(date, '2019-11-04')]
X_train, X_val = X.iloc[train_idxs, :], X.iloc[list(set(df.index) - set(train_idxs)), :]
y_train, y_val = y[train_idxs], y[list(set(df.index) - set(train_idxs))]
X_train.head()

In [ ]:
plt.figure(figsize = (10,7))

plt.hist(y_train, bins='auto', alpha=0.4, density = True, color='magenta', 
         label = 'Training set')
plt.hist(y_val, bins='auto', alpha=0.4, density = True, color='green', 
         label = 'Validation set')

plt.title('Comparison of BSRT demand distributions of training- and validation set', fontsize = 15)
plt.xlabel('Total demand')
plt.ylabel('Frequency')
plt.legend(fontsize = 12)
plt.show()

## Model selection

We fitted a wide variety of models to the data, and we found that forest-based estimators performed the best in terms of mean absolute error (MAE). Further, we refined this by choosing a forest of `ExtraTrees`, which also performs the splittings in the individual trees at random, reducing variance.

We performed a Bayesian hyperparameter search for 150 iterations over all the parameters in the forest, and found that the only parameter that resulted in better performance over the default values were the number of trees, which we increased from 100 to 3000.

In [ ]:
model = load_model_data()['model']
model

## Prediction intervals

We also wanted to have an idea of the uncertainty coupled with the predictions. We therefore wanted to estimate the variance of the predictions, to approximate both the sample noise and the noise within the trees themselves. A common way of doing this is by trainin the same forest on *bootstrapped samples*, which are datasets that are sampled from our original dataset with replacement, simulating "random datasets".

The problem with this is inference time: if we're using, say, 1000 bootstrapped datasets, then we need to apply each of our 3000 trees a thousand times every time we want to make a prediction.

A solution to this problem was found in the 2014 paper `Confidence intervals for random forests: The jackknife and the infinitesimal jackknife` by Wager, Hastie and Efron. They found a way to estimate the variance by *only using the trees once*.

For this to work we need to have trained our trees on bootstrapped samples, which reduced the performance of the model by a tiny bit (~1 MAE), but we are willing to sacrifice this small amount of performance to reduce the prediction time by a factor of 1000. We've implemented their method into the class `pExtraTreesRegressor`.

With their estimate of variance, we can then proceed to compute the standard error of the predictions. We're fortunate that our target values are normally distributed, which allows us to compute the standard prediction error as follows:

$$ \textsf{SE} = \textsf{Var}_b[\mathcal T_b(x)]^{1/2}\sqrt{1 + \tfrac{1}{B}}, $$

where $x$ is a new sample, $\mathcal T_b(x)$ is the prediction of the $b$'th tree, and $B$ is the amount of trees in the forest. We can therefore compute our $\alpha$ prediction intervals as

$$ \text{Prediction interval} = \mathbb{E}_b[\mathcal T_b(x)] \pm T_\alpha^{n-1}\textsf{SE}, $$

where $n$ is the number of training samples, and $T_\alpha^{n-1}$ is the $\tfrac{1 + \alpha}{2}$ percentile of the t-distribution with $n-1$ degrees of freedom.

As an example, if $\alpha = 95\%$ and we have 138 training samples then $T_\alpha^{n-1}\sim 1.98$. In general, as we have more training samples then this will converge to the familiar $1.96$ as the t-distribution converges to the normal distribution.

These are all implemented in the `predict` method of the `pExtraTreesRegressor` class, just set `return_intervals = True`.

## Visualising model performance

We now visualise the performance of the choice of model. We do this by restricting ourselves to a shorter period, and thereby also restricting the number of training samples, and then attempt to predict the following period. We choose here to show the 50%, 80%, 95% and 99% prediction intervals.

In [ ]:
alphas = [0.5, .8, .95, .99]
opacities = np.linspace(0.1, 0.4, len(alphas))

In [ ]:
def viz_model(cutoff_date = '2019-11-04'):
    train_idxs = [idx for idx, date in zip(df.index, df.date) if earlier_than(date, cutoff_date)]
    
    print(f'Number of training samples: {len(train_idxs)}')
    print(f'Number of validation samples: {len(df) - len(train_idxs)}')
    
    X_train, X_val = X.iloc[train_idxs, :], X.iloc[list(set(df.index) - set(train_idxs)), :]
    y_train, y_val = y[train_idxs], y[list(set(df.index) - set(train_idxs))]
    idxs = sorted(X_val.index)
    
    fig, ax = plt.subplots(figsize = (15, 7))
    plt.scatter(df.date[idxs], df.total[idxs], marker = 'x', label = 'True values', color = 'black')
    
    model = load_model_data()['model'].fit(X_train, y_train)
    for idx, alpha in enumerate(alphas):
        preds, intervals = model.predict(X_val, return_intervals = True, alpha = alpha)
        plt.fill_between(df.date[idxs], intervals[:, 0], intervals[:, 1], 
                     figure = fig, color = 'purple', alpha = opacities[-(idx + 1)], 
                         label = f'{round(100 * alpha, 1)}% prediction interval')
        
        coverage = sum((df.total[idxs] > intervals[:,0]) & (df.total[idxs] < intervals[:,1])) / len(df.total[idxs])
        print(f'Coverage for the {100 * alpha:.1f}% prediction interval: {100 * coverage:.1f}%')
        
    plt.legend(fontsize = 11)
    plt.xticks(rotation = 90)
    plt.title(f'Predictions - {type(model).__name__}', fontsize = 18)  
    
    plt.show()

In [ ]:
viz_model(cutoff_date = '2019-11-04')

As we can see, the model is not doing a fantastic job, which we attribute to the difference in the training- and validation distributions we saw above.

Note also that the actual model is better than this, as it's trained on more data.

## Using the model to impute data

In [ ]:
model.fit(X, y)
y_hat = model.predict(X)

In [ ]:
plt.figure(figsize = (10,7))

plt.hist(y, bins='auto', alpha=0.4, density = True, color='magenta', 
         label = 'True values')
plt.hist(y_hat, bins='auto', alpha=0.4, density = True, color='green', 
         label = 'Predicted values')

plt.title('Comparison of BSRT demand distributions of true and predicted values', fontsize = 15)
plt.xlabel('Total demand')
plt.ylabel('Frequency')
plt.legend(fontsize = 12)
plt.show()

In [ ]:
sorted(zip(model.feature_importances_, X.columns), reverse = True)

We can also use the (fully trained) model to impute all the missing values in our data set.

We have weather data for all the dates in `weather_df`, so we just need to extract the date features and merge them.

In [ ]:
dates = weather_df['date'].map(lambda x: datetime.strptime(x, '%Y-%m-%d'))
date_df = extract_date_data(dates)
date_df.head()

In [ ]:
feat_df = pd.concat([date_df, weather_df[['date'] + features]], axis = 1)
feat_df.head()

In [ ]:
feat_df['month'] = feat_df['month'].map(month)
feat_df['day_of_week'] = feat_df['day_of_week'].map(day_of_week)
feat_df.head()

We next load our model and perform all the predictions for the missing values.

In [ ]:
pred_df = feat_df.copy()
for alpha in alphas:
    preds, intervals = model.predict(feat_df.drop(columns = ['date']), 
                                     return_intervals = True, alpha = alpha)
    pred_df['prediction'] = preds
    pred_df[round(100 * (1 - alpha) / 2, 1)] = intervals[:, 0]
    pred_df[round(100 * ((1 - alpha) / 2 + alpha), 1)] = intervals[:, 1]
pred_df.head()

Using these predictions, let's predict the imputed values.

In [ ]:
fig = plt.figure(figsize = (40, 13))

for idx, alpha in enumerate(alphas):
    plt.fill_between(pred_df.date, 
                     pred_df[round(100 * (1 - alpha) / 2, 1)], 
                     pred_df[round(100 * ((1 - alpha) / 2 + alpha), 1)],
                     figure = fig, 
                     color = 'purple', 
                     alpha = opacities[-(idx + 1)],
                     label = f'{int(round(alpha * 100))}% prediction interval'
                     )
plt.scatter(df.date, df.total, color = 'green', marker = 'x', label = 'True values')
plt.xticks(range(1, len(pred_df.index), 25), 
           pred_df.date[list(range(1, len(pred_df.index), 25))], 
           rotation = 45)
plt.legend(fontsize = 17)
plt.title(f'Imputing total demand with {type(model).__name__}', fontsize = 30)
plt.show()

In [ ]:
for alpha in alphas:
    upper_val = round(100 * (1 + alpha) / 2, 1)
    lower_val = round(100 * (1 - alpha) / 2, 1)

    proj_preds = pred_df.copy()[pred_df.date.isin(df.date)]
    below_upper = df.total.values <= proj_preds[upper_val].values
    above_lower = df.total.values >= proj_preds[lower_val].values
    coverage = sum(below_upper & above_lower) / len(df)

    err_df = proj_preds.copy()
    err_df['total'] = df.total.values
    above_upper = err_df[~below_upper]
    above_err = max(0, (above_upper.total - above_upper[upper_val]).mean())
    below_lower = err_df[~above_lower]
    below_err = max(0, (below_lower[lower_val] - below_lower.total).mean())

    off_value = (above_err + below_err) / 2

    interval_size = (proj_preds[upper_val].values - proj_preds[lower_val].values).mean()

    print(f'{coverage * 100:.1f}% of the demand was within the {100 * alpha:.1f}% prediction interval.')
    print(f'The values outside the interval were on average {off_value:.2f} off.')
    print(f'The average length of the interval is {interval_size:.2f}.\n')

We see that the intervals capture most of the values, which should also be the case as those are the values that we trained the model on!

In [ ]:
ss = ShuffleSplit(n_splits = 5, test_size = 0.2)
pbar = tqdm(ss.split(range(X.shape[0])), desc = 'Evaluating', total = ss.n_splits)

maes, covs, offs, lens = np.empty(ss.n_splits), np.empty(ss.n_splits), np.empty(ss.n_splits), np.empty(ss.n_splits)
for idx, (train_idxs, test_idxs) in enumerate(pbar):
    model.fit(X.iloc[train_idxs, :], y[train_idxs])
    preds, intervals = model.predict(X.iloc[test_idxs, :], return_intervals = True, alpha = 0.99)
    
    below_upper = y[test_idxs] <= intervals[:, 1]
    above_lower = y[test_idxs] >= intervals[:, 0]
    coverage = np.mean(below_upper & above_lower)

    above_err = np.mean(y[test_idxs][~below_upper] - intervals[~below_upper, 1])
    below_err = np.mean(intervals[~above_lower, 0] - y[test_idxs][~above_lower])
    
    covs[idx] = np.mean((y[test_idxs] < intervals[:, 1]) & (y[test_idxs] > intervals[:, 0]))
    lens[idx] = np.mean(intervals[:, 1] - intervals[:, 0])
    maes[idx] = np.abs(preds - y[test_idxs]).mean()
    offs[idx] = (above_err + below_err) / 2
    
print(f'Mean absolute errors: {maes}')
print(f'\t- Average: {maes.mean()}')
print(f'\t- Standard deviation: {maes.std()}')

print(f'\nLengths: {covs}')
print(f'\t- Average: {covs.mean()}')
print(f'\t- Standard deviation: {covs.std()}')

print(f'\nCoverages: {lens}')
print(f'\t- Average: {lens.mean()}')
print(f'\t- Standard deviation: {lens.std()}')

print(f'\nOff values: {offs}')
print(f'\t- Average: {offs.mean()}')
print(f'\t- Standard deviation: {offs.std()}')